In [10]:
""" importing necessary packages"""

import pandas as pd
import numpy as np

In [15]:
"""
Getting the raw csv file (the column -name was split into Company and Car in MSExcel)
"""

carprice_data = pd.read_csv("C:\\Users\\U6080693\\OneDrive - London Stock Exchange Group\\Desktop\\Mini Hackathon\\data\\Car_price_data.csv").drop(columns=["name"])

In [16]:
""" viewing random samples """
carprice_data.sample(3)

,Company,Car,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,selling_price
3320,Ford,Ford Figo Diesel,2011,80000,Diesel,Individual,Manual,First Owner,20.0 kmpl,1399 CC,68 bhp,160Nm@ 2000rpm,5.0,300000
6272,Hyundai,Hyundai i20 2015-2017,2017,28782,Petrol,Dealer,Manual,First Owner,18.6 kmpl,1197 CC,81.83 bhp,114.7Nm@ 4000rpm,5.0,570000
6421,Maruti,Maruti Wagon R,2020,70000,Petrol,Individual,Manual,Second Owner,21.79 kmpl,998 CC,67.05 bhp,90Nm@ 3500rpm,5.0,265000


In [ ]:
""" information on columns """
carprice_data.info()

In [ ]:
""" inspecting numerical variables """
carprice_data.describe()

In [17]:
"""
inspecting a -Car value to see the values of mileage, engine, seats and max_power
the values are unique for a Car model. This can be used to impute missing values
"""

carprice_data[carprice_data['Car']=="Ford Figo Diesel"]

,Company,Car,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,selling_price
9,Ford,Ford Figo Diesel,2013,169000,Diesel,Individual,Manual,First Owner,20.0 kmpl,1399 CC,68.1 bhp,160Nm@ 2000rpm,5.0,200000
173,Ford,Ford Figo Diesel,2011,120000,Diesel,Individual,Manual,First Owner,20.0 kmpl,1399 CC,68 bhp,160Nm@ 2000rpm,5.0,160000
344,Ford,Ford Figo Diesel,2010,90000,Diesel,Individual,Manual,Second Owner,20.0 kmpl,1399 CC,68 bhp,160Nm@ 2000rpm,5.0,170000
446,Ford,Ford Figo Diesel,2015,70000,Diesel,Individual,Manual,First Owner,20.0 kmpl,1399 CC,68.05 bhp,160Nm@ 2000rpm,5.0,292000
581,Ford,Ford Figo Diesel,2013,70000,Diesel,Individual,Manual,Second Owner,20.0 kmpl,1399 CC,68.1 bhp,160Nm@ 2000rpm,5.0,245000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7504,Ford,Ford Figo Diesel,2013,104300,Diesel,Individual,Manual,First Owner,20.0 kmpl,1399 CC,68.1 bhp,160Nm@ 2000rpm,5.0,275000
7619,Ford,Ford Figo Diesel,2010,70000,Diesel,Individual,Manual,First Owner,20.0 kmpl,1399 CC,68 bhp,160Nm@ 2000rpm,5.0,260000
7889,Ford,Ford Figo Diesel,2011,90000,Diesel,Individual,Manual,Third Owner,20.0 kmpl,1399 CC,68 bhp,160Nm@ 2000rpm,5.0,180000
8012,Ford,Ford Figo Diesel,2011,170000,Diesel,Individual,Manual,Second Owner,20.0 kmpl,1399 CC,68 bhp,160Nm@ 2000rpm,5.0,200000


In [4]:
"""
Finding the missing values features and replacing them with NaN 

Missing values accounts to less than 3% of samples, these can be imputed
"""

def missing_value_pct(data : pd.DataFrame) -> pd.DataFrame :

    missing_pct = (((data.isna().sum())/data.shape[0])*100).reset_index(name="percent_missing")
    return missing_pct.rename(columns= {"index": "column_name"}).query('percent_missing>0')

missing_pct = missing_value_pct(carprice_data)
print(missing_pct)

   column_name  percent_missing
8      mileage         2.718996
9       engine         2.718996
10   max_power         2.645177
11      torque         2.731299
12       seats         2.718996


In [5]:
""" removing units in the entries and renaming columns """

carprice_data["engine"].replace("[A-Za-z]+","", regex=True, inplace=True)
carprice_data["max_power"].replace("[A-Za-z]+","", regex=True, inplace=True)

carprice_data.rename(columns= {"engine": "engine_cc", "max_power":"max_power_bhp"}, inplace=True)

In [6]:
"""
mileage has 2 units - kmpl and km/kg, extracting the numbers now and later will convert them with appropriate transformation (also removing trailing spaces)
"""

carprice_data["mileage"] = carprice_data["mileage"].str.extract(r'([0-9.]+)')
carprice_data['max_power_bhp']=carprice_data['max_power_bhp'].str.strip()
carprice_data['max_power_bhp'].replace('',np.nan, regex=True,inplace=True)

carprice_data.sample(3)


,Company,Car,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine_cc,max_power_bhp,torque,seats,selling_price
7253,Chevrolet,Chevrolet Beat Diesel,2011,70000,Diesel,Individual,Manual,Second Owner,25.44,936,57.6,150Nm@ 1750rpm,5.0,165000
2162,Maruti,Maruti Alto K10,2012,48000,Petrol,Individual,Manual,First Owner,20.92,998,67.1,90Nm@ 3500rpm,5.0,252000
4126,Maruti,Maruti Alto K10,2017,50000,Petrol,Individual,Manual,First Owner,23.95,998,67.05,90Nm@ 3500rpm,5.0,330000


In [7]:
""" fill all the empty cells with NaN, so the columns can be type-casted """

carprice_data.fillna(np.nan, inplace=True)

In [8]:
""" type casting columns to relevent datatype """

carprice_data = carprice_data.astype( { 'km_driven': 'float',
                                        'mileage': 'float',
                                        'engine_cc': 'float',
                                        'max_power_bhp': 'float',
                                        'seats': 'Int64',
                                        'selling_price': 'float'}

)

In [9]:
carprice_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        8128 non-null   object 
 1   Car            8128 non-null   object 
 2   year           8128 non-null   int64  
 3   km_driven      8128 non-null   float64
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   float64
 9   engine_cc      7907 non-null   float64
 10  max_power_bhp  7912 non-null   float64
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   Int64  
 13  selling_price  8128 non-null   float64
dtypes: Int64(1), float64(5), int64(1), object(7)
memory usage: 897.1+ KB


In [11]:
carprice_data.drop(columns=['torque'], inplace=True)

,Company,Car,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine_cc,max_power_bhp,seats,selling_price
0,Maruti,Maruti Swift Dzire,2014,145500.0,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,5,450000.0
1,Skoda,Skoda Rapid 1.5,2014,120000.0,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5,370000.0
2,Honda,Honda City 2017-2020,2006,140000.0,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78.00,5,158000.0
3,Hyundai,Hyundai i20 Sportz,2010,127000.0,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,5,225000.0
4,Maruti,Maruti Swift VXI,2007,120000.0,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,5,130000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai,Hyundai i20 Magna,2013,110000.0,Petrol,Individual,Manual,First Owner,18.50,1197.0,82.85,5,320000.0
8124,Hyundai,Hyundai Verna CRDi,2007,119000.0,Diesel,Individual,Manual,Fourth & Above Owner,16.80,1493.0,110.00,5,135000.0
8125,Maruti,Maruti Swift Dzire,2009,120000.0,Diesel,Individual,Manual,First Owner,19.30,1248.0,73.90,5,382000.0
8126,Tata,Tata Indigo CR4,2013,25000.0,Diesel,Individual,Manual,First Owner,23.57,1396.0,70.00,5,290000.0


In [12]:
carprice_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        8128 non-null   object 
 1   Car            8128 non-null   object 
 2   year           8128 non-null   int64  
 3   km_driven      8128 non-null   float64
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   float64
 9   engine_cc      7907 non-null   float64
 10  max_power_bhp  7912 non-null   float64
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   Int64  
 13  selling_price  8128 non-null   float64
dtypes: Int64(1), float64(5), int64(1), object(7)
memory usage: 897.1+ KB


In [34]:
carprice_data.to_csv("C:\\Users\\U6080693\\OneDrive - London Stock Exchange Group\\Desktop\\Mini Hackathon\\data\\carprice_data_cleaned.csv", index=False)